In [1]:
!git clone https://github.com/karen-pal/borges

Cloning into 'borges'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 211 (delta 89), reused 171 (delta 49), pack-reused 0
Receiving objects: 100% (211/211), 2.21 MiB | 13.89 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [2]:
pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255315 sha256=827014ac645b1c38ef51dcb10bdd7410dfcd5f76bd8889c4a7c4e990d17ce0b5
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [3]:
import pickle
from pathlib import Path
import pandas as pd

In [4]:
df = pd.DataFrame()
# usando el asterisco de "wildcard" traemos todos los archivos en formato pickle
pkls = Path('.').glob('./borges/datasets/*texts.pkl')

# leemos todos los pickles y concatenarlos en un DataFrame
for pkl in pkls:
    with open(pkl, 'rb') as inp:
        df_ = pickle.load(inp)
    df = pd.concat([df, df_])

df.shape

(719, 3)

In [5]:
df = df.drop_duplicates(subset=[c for c in df.columns if c != 'text_metadata'])
df = df.reset_index(drop=True)
df.shape

(693, 3)

In [6]:
df['title'] = df['text_metadata'].apply(lambda x: x['title'])
df['author'] = df['text_metadata'].apply(lambda x: x['author'])

In [7]:
df.head(4)

,link,text_metadata,text,title,author
0,https://ciudadseva.com/texto/el-antropofago/,"{'title': 'El antropófago', 'metadata': '[Cuen...","Allí está, en la penitenciaría, asomando por e...",El antropófago,Pablo Palacio
1,https://ciudadseva.com/texto/la-doble-y-unica-...,"{'title': 'La doble y única mujer', 'metadata'...",(Ha sido preciso que me adapte a una serie de ...,La doble y única mujer,Pablo Palacio
2,https://ciudadseva.com/texto/revolucion-palacio/,"{'title': 'Revolución', 'metadata': '[Minicuen...","Pesas, pesas tanto. Pues salta sobre un platil...",Revolución,Pablo Palacio
3,https://ciudadseva.com/texto/senora-palacio/,"{'title': 'Señora', 'metadata': '[Cuento - Tex...","–Usted fue, sí, usted fue. –¿Señora…? –Le digo...",Señora,Pablo Palacio


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from pprint import pprint
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = list(stopwords.words('spanish'))
# eliminamos las "stop words", palabras comunes no informativas
tf = TfidfVectorizer(stop_words=stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
import random
keywords = [
    "Desarrollador de UI",
    "Científico de datos",
    "Ingeniero de software",
    "Desarrollador móvil",
    "Ingeniero de seguridad informática",
    "Analista de sistemas",
    "Desarrollador front-end",
    "Desarrollador back-end",
    "Arquitecto de software",
    "Ingeniero de redes",
    "Especialista en aprendizaje automático",
    "Ingeniero de sistemas embebidos",
    "Desarrollador de videojuegos",
    "Desarrollador de realidad virtual",
    "Desarrollador de aplicaciones web",
    "Ingeniero de calidad de software",
    "Especialista en automatización de pruebas",
    "Analista de datos",
    "Desarrollador de blockchain",
    "Ingeniero de DevOps"
]

for i in range(len(df)):
    # Generamos una palabra clave aleatoria
    keyword = keywords[random.randint(0, len(keywords) - 1)]
    # Agregamos la palabra clave a la fila
    df.loc[i, "Palabra clave"] = keyword

print(df)

                                                  link  \
0         https://ciudadseva.com/texto/el-antropofago/   
1    https://ciudadseva.com/texto/la-doble-y-unica-...   
2     https://ciudadseva.com/texto/revolucion-palacio/   
3         https://ciudadseva.com/texto/senora-palacio/   
4    https://ciudadseva.com/texto/un-hombre-muerto-...   
..                                                 ...   
688  https://ciudadseva.com/texto/tlon-uqbar-orbis-...   
689  https://ciudadseva.com/texto/tres-versiones-de...   
690          https://ciudadseva.com/texto/un-problema/   
691           https://ciudadseva.com/texto/un-sueno-2/   
692  https://ciudadseva.com/texto/utopia-de-un-homb...   

                                         text_metadata  \
0    {'title': 'El antropófago', 'metadata': '[Cuen...   
1    {'title': 'La doble y única mujer', 'metadata'...   
2    {'title': 'Revolución', 'metadata': '[Minicuen...   
3    {'title': 'Señora', 'metadata': '[Cuento - Tex...   
4    {'title'

In [11]:
# calculamos los features para cada ítem (texto y gustos)
tfidf_text = tf.fit_transform(df['text'])
tfidf_gustos = tf.transform(df["Palabra clave"])

In [12]:
cosine = cosine_similarity(tfidf_text,tfidf_text)
cosine2 = cosine_similarity(tfidf_gustos, tfidf_gustos)
n = 6

result = cosine_similarity(cosine,cosine2)

print(result)

''' print(cosine) '''
''' print(kernel) '''
''' print(epico) '''

[[0.         0.16226028 0.18443355 ... 0.         0.31844079 0.13992744]
 [0.         0.24415324 0.2294901  ... 0.         0.38197665 0.17574735]
 [0.         0.04870466 0.20311989 ... 0.         0.10996655 0.06146484]
 ...
 [0.         0.08094989 0.10486559 ... 0.         0.20050322 0.06580247]
 [0.         0.07988934 0.0839281  ... 0.         0.2640234  0.09009743]
 [0.         0.1675367  0.19909593 ... 0.         0.37538985 0.24986211]]


' print(epico) '

In [13]:
results = {}
for idx, row in df.iterrows():
    # guardamos los indices similares basados en la similitud coseno. Los ordenamos en modo ascendente, siendo 0 nada de similitud y 1 total
    similar_indices = result[idx].argsort()[:-n-2:-1]
    # guardamos los N más cercanos
    similar_items = [(f"{df['author'][i]} - {df['title'][i]}", round(result[idx][i], 3)) for i in similar_indices]
    results[f"{row['author']} - {row['title']}"] = similar_items[1:]

In [14]:
pprint(results['Manuel A. Alonso - El jíbaro en la capital'])

[('Mario Benedetti - Cleopatra', 0.38),
 ('José María Arguedas - La muerte de los Arango', 0.38),
 ('Rubén Darío - Huitzilopoxtli', 0.38),
 ('Álvaro Mutis - Sueño del fraile', 0.38),
 ('Andrés Rivera - Bailé', 0.38),
 ('Baldomero Lillo - Irredención', 0.38)]


In [15]:
def recomendar(autor, titulo):
    pprint(results[f"{autor} - {titulo}"])

In [16]:
recomendar('Santiago Dabove', 'El tren')

[('Augusto Monterroso - El grillo maestro', 0.333),
 ('Enrique Anderson Imbert - Sententia Nominum', 0.333),
 ('Rubén Darío - Amar hasta fracasar (solo la vocal “a”)', 0.333),
 ('Andrés Rivera - El corrector', 0.333),
 ('Manuel Romero de Terreros - La puerta de bronce', 0.333),
 ('Roberto Arlt - Los hombres fieras', 0.333)]
